# Imports

In [1]:
import numpy as np
import pandas as pd

# Datas

In [2]:
# Load colums in dataframes for each dataset
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

Train shape: (1790594, 17)
Test shape: (146853, 3)


In [3]:
# Info about datas
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790594 entries, 0 to 1790593
Data columns (total 17 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   StudyInstanceUID            object
 1   SeriesInstanceUID           object
 2   SOPInstanceUID              object
 3   pe_present_on_image         int64 
 4   negative_exam_for_pe        int64 
 5   qa_motion                   int64 
 6   qa_contrast                 int64 
 7   flow_artifact               int64 
 8   rv_lv_ratio_gte_1           int64 
 9   rv_lv_ratio_lt_1            int64 
 10  leftsided_pe                int64 
 11  chronic_pe                  int64 
 12  true_filling_defect_not_pe  int64 
 13  rightsided_pe               int64 
 14  acute_and_chronic_pe        int64 
 15  central_pe                  int64 
 16  indeterminate               int64 
dtypes: int64(14), object(3)
memory usage: 232.2+ MB


In [4]:
# Some examples 
df_train.head(n=10)

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate
0,6897fa9de148,2bfbb7fd2e8b,c0f3cb036d06,0,0,0,0,0,0,1,1,0,0,1,0,0,0
1,6897fa9de148,2bfbb7fd2e8b,f57ffd3883b6,0,0,0,0,0,0,1,1,0,0,1,0,0,0
2,6897fa9de148,2bfbb7fd2e8b,41220fda34a3,0,0,0,0,0,0,1,1,0,0,1,0,0,0
3,6897fa9de148,2bfbb7fd2e8b,13b685b4b14f,0,0,0,0,0,0,1,1,0,0,1,0,0,0
4,6897fa9de148,2bfbb7fd2e8b,be0b7524ffb4,0,0,0,0,0,0,1,1,0,0,1,0,0,0
5,6897fa9de148,2bfbb7fd2e8b,c7b99cb454d4,0,0,0,0,0,0,1,1,0,0,1,0,0,0
6,6897fa9de148,2bfbb7fd2e8b,e153deb813ed,0,0,0,0,0,0,1,1,0,0,1,0,0,0
7,6897fa9de148,2bfbb7fd2e8b,c6bbe08f2736,0,0,0,0,0,0,1,1,0,0,1,0,0,0
8,6897fa9de148,2bfbb7fd2e8b,09886998dc28,0,0,0,0,0,0,1,1,0,0,1,0,0,0
9,6897fa9de148,2bfbb7fd2e8b,5b5226ac7bac,0,0,0,0,0,0,1,1,0,0,1,0,0,0


In [5]:
# Pack dependant features in one categorical feature

# Merge 2 ratio features in a single one
ratio_mask = [x == 1 for x in df_train['rv_lv_ratio_gte_1']]
df_train['rv_lv_ratio'] = pd.Categorical(np.where(ratio_mask, ">=1", "<1"))

# Merge pe type related features in a signle one

# Take chronic and acute+chronic
def categorize_pe_type(data):
    if data['chronic_pe']: return "Chronic"
    elif data['acute_and_chronic_pe']: return "Acute and Chronic"
    else: return "Acute"
    
    
df_train['pe_type'] = pd.Categorical(df_train.apply(categorize_pe_type, axis=1))

In [6]:
# Remove packed features

# TODO add asserts to ensure a good behavior before dropping
df_train.drop(['rv_lv_ratio_gte_1','rv_lv_ratio_lt_1'], axis=1, inplace=True)
df_train.drop(['chronic_pe','acute_and_chronic_pe'], axis=1, inplace=True)

In [7]:
# Show dataframe's info with new features
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790594 entries, 0 to 1790593
Data columns (total 15 columns):
 #   Column                      Dtype   
---  ------                      -----   
 0   StudyInstanceUID            object  
 1   SeriesInstanceUID           object  
 2   SOPInstanceUID              object  
 3   pe_present_on_image         int64   
 4   negative_exam_for_pe        int64   
 5   qa_motion                   int64   
 6   qa_contrast                 int64   
 7   flow_artifact               int64   
 8   leftsided_pe                int64   
 9   true_filling_defect_not_pe  int64   
 10  rightsided_pe               int64   
 11  central_pe                  int64   
 12  indeterminate               int64   
 13  rv_lv_ratio                 category
 14  pe_type                     category
dtypes: category(2), int64(10), object(3)
memory usage: 181.0+ MB


In [11]:
# Show samples 
df_train.head(n=10)

0            Acute
1            Acute
2            Acute
3            Acute
4            Acute
            ...   
1790589    Chronic
1790590    Chronic
1790591    Chronic
1790592    Chronic
1790593    Chronic
Name: pe_type, Length: 1790594, dtype: category
Categories (3, object): [Acute, Acute and Chronic, Chronic]